In [9]:
import utils


In [ ]:
# get iceberg runtime
spark_version = "3.5"
scala_version = "2.12"
iceberg_version = "1.7.0"
target_dir = "~/spark_jars/iceberg"

jar_path = utils.download_iceberg_jar(spark_version, scala_version, iceberg_version, target_dir)
print(f"JAR saved at: {jar_path}")

In [11]:
from pyspark.sql import SparkSession

catalog_name = "iceberg"

# Initialize Spark session with Iceberg configuration
spark = SparkSession.builder \
    .appName("IcebergLocalExample") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", "./icehouse") \
    .config("spark.jars", jar_path) \
    .getOrCreate()


In [12]:
data = [(1, "Alice", "2024-11-28 12:00:00"),
        (2, "Bob", "2024-11-28 13:00:00")]
columns = ["id", "name", "ts"]

df = spark.createDataFrame(data, columns)

table_name = "test_tbl2"

# # Write the DataFrame to Iceberg
df.writeTo(f"{catalog_name}.default.{table_name}") \
    .using("iceberg") \
    .tableProperty("write.format.default", "parquet") \
    .createOrReplace()

